# How to:
## Visuzlize Jackknifed Measurement Sets
This tutorial describes how to jack knife measurement sets (ms-files) -- in particular ALMA data -- to create observation-specific noise realizations. We use the `jacked` package to do the interferace with ms-file, split it, and image the visibillities.

In [1]:
import jacked

We need to initialize the measurement set, so that python can do the interface.

In [2]:
#initilaize
tool = jacked.Jack(fname   = '../data/Glass-z13.ms', # The File name of the visibilities, 
                   outdir  = '../output/' 
                   fields  = ['3'], # Each visibility can have multiple fields, 
                   spws    = [['0']], # and each field can have multiple spws,
                   band    = 'Band7', # Band of the observation,
                   array   = 'C7', # which configuration the observation are taken in,
                   )

TypeError: __init__() missing 1 required positional argument: 'outdir'

To creat a Jack knifed measurement set, you simply need to run:

In [ ]:
tool.run(seed = 42)

Want another one? Just run it again, but with another seed:

In [3]:
# tool.run(seed = 142)

  0%|          | 0/1 [00:00<?, ?it/s]

.. Loading in MS
.. Jack Knife it
.. Saving to MS
.. Image


  0%|          | 0/1 [03:23<?, ?it/s]
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/jvanmarr/opt/anaconda3/envs/jacked_env/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-4-760429fedd64>", line 1, in <module>
    tool.run(seed = 142)
  File "/Users/jvanmarr/Documents/GitHub/Jack-knife/jacked/Operator.py", line 146, in run
  File "/Users/jvanmarr/Documents/GitHub/Jack-knife/jacked/Operator.py", line 117, in clean
    weighting   =                            'natural',
  File "/Users/jvanmarr/opt/anaconda3/envs/jacked_env/lib/python3.6/site-packages/casatasks/tclean.py", line 1717, in __call__
    task_result = _tclean_t( _pc.document['vis'], _pc.document['selectdata'], _pc.document['field'], _pc.document['spw'], _pc.document['timerange'], _pc.document['uvrange'], _pc.document['antenna'], _pc.document['scan'], _pc.document['observation'], _pc.document['intent'], _pc.document['datacolumn'], _pc.document['imag

TypeError: object of type 'NoneType' has no len()

Or if you want multiple simulatiously, just run:

In [ ]:
tool.run(samples = 3, seed = 242)

Done :)

## Image the data

To double check if everything went correctly, we build in some original plotting functions you can use to visualize the data.

In [ ]:
out_file = tool.vis_jacked.replace('.ms','.im.fits') # get the file name of the created noise cube

In [ ]:
from casatasks import tclean, exportfits, immoments

from jacked.ImSettings import Imparams
Imcase = Imparams(config = 'C7', band = 'Band7') # automatically defines the image size and the cell size

In [ ]:
path_to_data = '../data/'
path_to_output = '../output/'
name_of_obss = 'Glass-z13_target_concat_tbin30s_cwidth38MHz_60spw.ms'

outfile = path_to_output + name_of_obss

tclean(vis         = path_to_data + name_of_obss,
       imagename   = outfile.replace('.ms', '.im'),
       specmode    = 'cube',
       deconvolver = 'hogbom',
       gridder     = 'standard',
       imsize      = Imcase.imsize,
       cell        = Imcase.cellsize,
       reffreq     = '254.35GHz',
       weighting   = 'natural',
       niter       = 0,
       interactive = False,
       )

exportfits(imagename   = outfile.replace('.ms', '.im.image'),
           fitsimage   = outfile.replace('.ms', '.im.fits'),
           overwrite   = True
          )

In [ ]:
fname = outfile.replace('.ms', 'line.im.fits')
hdu = fits.open(outfile.replace('.ms', 'line.im.fits'))
cube = hdu[0].data[0]

In [ ]:
freqs = np.arange(len(cube))*hdu[0].header['CDELT3'] + hdu[0].header['CRVAL3']
velos = (freqs-254.35e9)/254.35e9*const.c.to(u.km/u.s).value

index = np.argmin(np.abs(velos))
moment0 = np.sum(cube[index-8:index+8], axis = 0)

# Image Data

## Moment-0

In [ ]:
def plot(fits_file, box_size_arcsec, center_coord, index):
    """
    Plots a FITS file image with WCS coordinates for the axes in RA and DEC.
    Allows zooming in to a specified box size in arcseconds centered on a given coordinate.

    Parameters:
    fits_file (str): Path to the FITS file.
    box_size_arcsec (float): Size of the box to display in arcseconds.
    center_coord (tuple): (RA, DEC) coordinates to center the zoom-in region.
    """
    # Open the FITS file
    with fits.open(fits_file) as hdul:
        # Get the data from the first HDU
        data = hdul[0].data
        data = np.sum(data[0,index[0]:index[1]], axis=0)*1e3
        
        # Get the WCS information from the header
        wcs = WCS(hdul[0].header, naxis = 2)
        
        beam_major = np.average(hdu[1].data['BMAJ'])  # arcseconds
        beam_minor = np.average(hdu[1].data['BMIN'])  # arcseconds
        beam_pa = np.average(hdu[1].data['BPA'])  # Position angle in degrees

    
    # Convert the center coordinates from (RA, DEC) to pixel values
    center_x, center_y = wcs.world_to_pixel_values(center_coord[0], center_coord[1])
    
    # Convert box size from arcseconds to pixels
    pix_scale_x = np.abs(wcs.wcs.cdelt[0]) * 3600  # arcsec per pixel in x direction
    pix_scale_y = np.abs(wcs.wcs.cdelt[1]) * 3600  # arcsec per pixel in y direction
    
    box_size_x_pix = box_size_arcsec / pix_scale_x
    box_size_y_pix = box_size_arcsec / pix_scale_y
    
    # Determine the bounding box in pixel coordinates
    x_min = int(center_x - box_size_x_pix / 2)
    x_max = int(center_x + box_size_x_pix / 2)
    y_min = int(center_y - box_size_y_pix / 2)
    y_max = int(center_y + box_size_y_pix / 2)
    
    # Estimate the standard deviation outside the bounding box
    mask = np.ones(data.shape, dtype=bool)
    mask[y_min:y_max, x_min:x_max] = False
    std_dev = np.nanstd(data[mask])

    # Crop the data to the bounding box
    cropped_data = data[y_min:y_max, x_min:x_max]

    # Adjust WCS for the cropped image
    cropped_wcs = wcs[y_min:y_max, x_min:x_max]

    # Plot the cropped image with WCS projection
    fig, ax = plt.subplots(subplot_kw={'projection': cropped_wcs}, figsize=(10,8))
    im = ax.imshow(data[y_min:y_max, x_min:x_max], origin='lower', cmap='RdBu_r', vmin = np.nanmin(data), vmax = -1*np.nanmin(data))
    ax.set_xlabel('RA (J2000)')
    ax.set_ylabel('DEC (J2000)')
    
    # Plot contours based on the estimated standard deviation
    levels = [-3*std_dev, -2*std_dev,-1*std_dev, std_dev, 2*std_dev, 3*std_dev]  # Example contour levels
    ax.contour(cropped_data, levels=levels, colors='k', alpha=0.7)
    
    # Add a colorbar
    cbar = plt.colorbar(im, ax=ax, orientation='vertical')
    cbar.set_label(r'$S_{\nu}$ [mJy/Beam km/s]')

    # Plot the beam size in the lower right corner
    beam_major_pix = beam_major / pix_scale_x
    beam_minor_pix = beam_minor / pix_scale_y    
    beam = Ellipse((10, 10), width=beam_minor_pix, height=beam_major_pix,
                   angle=beam_pa, color='gray', fill='//')
    ax.add_patch(beam)
    
    # Plot a cross at the center coordinates
    c = SkyCoord(center_coord[0], center_coord[1], frame='icrs', unit='deg')
    center_x, center_y = cropped_wcs.world_to_pixel(c)
    
    ax.scatter(center_x, center_y, color='goldenrod', marker ='x', s = 100)
    # ax.scatter(center_coord[0], center_coord[1], color='goldenrod', marker ='x')
    plt.show()

In [ ]:
fname = '../output/cleaned/Glass-z13_target_concat_tbin30s_cwidth38MHz_60spw_contsubline.im.fits'
plot_fits_image_with_wcs(fname, 4, (3.498985, -30.324767), (index-5, index+5))  # Replace with your FITS file path, desired box size in arcseconds, and center coordinates (RA, DEC)

In [ ]:
fname = '../output/cleaned/Glass-z13_target_concat_tbin30s_cwidth38MHz_60spwline.im.fits'
plot_fits_image_with_wcs(fname, 4, (3.498985, -30.324767),  (index-5, index+5))  # Replace with your FITS file path, desired box size in arcseconds, and center coordinates (RA, DEC)

## SLP

In [ ]:
class SLP():
    """
    A class that manages cleaned data cubes and estiamtes the spectral line profile of that cube given a mask to sum over
    Input:
        cube (channel, image, image): Cleaned data in units of Jy/pixel
        mask (image, image): A mask that matches the size of the image. The mask is used to some over the pixels.
    Output (with function execute):
        slp (len(channels)): Intensity, units of Jy.
        bootstrap_std: bootstrapped std in units of Jy. 
    
    """
    
    def __init__(self, cube, mask, amount = 200, visualize = False):
        self.visualize = visualize
        self.cube   = cube
        self.mask   = mask
        self.amount = amount
        
        xx, yy         = np.meshgrid(np.arange(0,self.cube.shape[-2], 1), np.arange(0, self.cube.shape[-1],1))
        self.CoM_mask  =  (np.mean(xx[self.mask]), np.mean(yy[self.mask]))
        self.rr        = ((xx-self.CoM_mask[0])**2 + (yy-self.CoM_mask[1])**2)**0.5
        
        self.mask_size = int(np.sum(self.mask)**0.5+0.1*self.cube.shape[-2]) # --> quick fix

    def _position(self):

        r =  np.sqrt(np.random.uniform(0.1,1, size = self.amount))
        theta = np.random.uniform(0,1, size = self.amount) * 2 * np.pi

        x = self.CoM_mask[0] + self.cube.shape[1]/3 *r* np.cos(theta) #hard coded
        y = self.CoM_mask[1] + self.cube.shape[1]/3 *r* np.sin(theta) #hard coded
        
        pos = np.array([x,y], dtype = np.int).T
        return np.vstack(([int(self.CoM_mask[0]), int(self.CoM_mask[1])], pos))

    def _make_mask(self, pos):
        new_masks = np.zeros((len(pos[1:]),self.im.shape[-2], self.im.shape[-1]))
        for idx, xy in enumerate(pos[1:]):
            new_masks[idx, 
                      xy[0]-self.mask_size//2:xy[0]+self.mask_size//2, 
                      xy[1]-self.mask_size//2:xy[1]+self.mask_size//2] = self.mask[pos[0][0]-self.mask_size//2:pos[0][0]+self.mask_size//2, 
                                                                                   pos[0][1]-self.mask_size//2:pos[0][1]+self.mask_size//2]
            
            
        return new_masks.astype(bool)  
    
    def _estimates(self, new_masks):
        estimates = []
        for m in new_masks:
            estimates.append(np.sum(self.im[m]))
        return np.nanstd(estimates), np.nanmean(estimates)
    
    def _visualize(self, masks):
        for idx, m in enumerate(masks):
            clear_output(True)
            plt.imshow(m, origin='lower')
            plt.show()
            if idx>30: break
    
    def execute(self):

        bootstrap_std = []
        bootstrap_means = []
        for i in tqdm(range(len(self.cube))):
            self.im = self.cube[i]
            pos     = self._position()
            masks   = self._make_mask(pos)            

            std, mean = self._estimates(masks)
            bootstrap_std.append(std)
            bootstrap_means.append(mean)
            
        if self.visualize: self._visualize(masks)

        return np.array(bootstrap_std), np.array(bootstrap_means)

In [ ]:
def circle_mask(im, xc, yc, rcirc):
        ny, nx = im.shape
        y,x = np.mgrid[0:nx,0:ny]
        r = np.sqrt((x-xc)*(x-xc) + (y-yc)*(y-yc))
        return ( (r < rcirc))

def ellipsoid_mask(im, xc, yc, a, b, Theta):
    ny, nx = im.shape
    y,x = np.mgrid[0:nx,0:ny]
    e = (((x-xc) * np.cos(Theta) + (y - yc)*np.sin(Theta))**2)/a**2 +  (((x-xc) * np.sin(Theta) - (y - yc)*np.cos(Theta))**2)/b**2
    return (e<=1)

In [ ]:
# fname = '../output/cleaned/Glass-z13_target_concat_tbin30s_cwidth38MHz_60spwline.im.fits'

# # Load in Cube
# # ------------
# cube, header = fits.getdata(fname, header = True)

# BMAJ = np.average(hdu[1].data['BMAJ'])/3600  # degrees
# BMIN = np.average(hdu[1].data['BMIN'])/3600  # degrees
# BPA = np.average(hdu[1].data['BPA'])  # Position angle in degrees

# pixel_size = header['CDELT1']
# Dfreq    = header['CDELT3']
# restfreq = header['CRVAL3']

# BeamArea     = np.pi *BMIN*BMAJ/pixel_size**2/(4*np.log(2))   # pixels per beam 
# cube         = cube[0]/BeamArea*1e3                           # mJy/pixel

# mask = ellipsoid_mask(cube[0], 
#                       len(cube[0])//2, 
#                       len(cube[0])//2, 
#                       BMIN/pixel_size, 
#                       BMAJ/pixel_size, 
#                       np.deg2rad(BPA))


# xaxis      = (np.arange(len(cube))*Dfreq + restfreq)/1e9    
# slp        = np.nansum(cube[:, mask], axis = 1)
# normal_std = np.nanstd(cube[:,~mask], axis = 1) * np.sqrt(np.sum(mask))
# mask = ellipsoid_mask(cube[0], 
#                       len(cube[0])//2, 
#                       len(cube[0])//2, 
#                       BMIN/pixel_size, 
#                       BMAJ/pixel_size, 
#                       np.deg2rad(BPA))

# obj                           = SLP(cube, mask, amount = 1000, visualize=False)
# bootstrap_std, bootstrap_mean = obj.execute()

# # Visualize SLP
# # -----------------
# fig, ax = plt.subplots(2,1, sharex=True, figsize=(8,6), gridspec_kw={'height_ratios': [2, 1]})

# ax[0].step(xaxis, slp/bootstrap_std, label = 'Central Beam', c = 'C0')
# ax[0].axhline(0, c='gray', ls='--')
# ax[0].set_xlim(xmin = xaxis[0], xmax = xaxis[-1])
# ax[0].set_ylim(ymin =-2.5, ymax = 3.5)
# ax[0].set_ylabel('SNR')
# ax[0].legend(loc = 1, frameon=False)
# ax[0].axvline(254.35, c = 'k')

# # ax[1].step(xaxis, normal_std, label='conventional image')
# ax[1].step(xaxis, bootstrap_std, label='bootstrap', c = 'C1')
# ax[1].axhline(np.nanstd(slp), label='std spectral', c='k', alpha = 0.6)    

# ax[1].set_xlim(xmin = xaxis[0], xmax = xaxis[-1])
# ax[1].set_ylim(ymin = 0.15, ymax = 0.25)
# ax[1].set_xlabel('Freq [GHz]')
# ax[1].set_ylabel('mJy')
# ax[1].legend(loc = 1, frameon=False)

# plt.tight_layout()
# plt.show()

In [ ]:
# fname = '../output/cleaned/Glass-z13_target_concat_tbin30s_cwidth38MHz_60spw_contsubline.im.fits'

# # Load in Cube
# # ------------
# cube, header = fits.getdata(fname, header = True)

# BMAJ = np.average(hdu[1].data['BMAJ'])/3600  # degrees
# BMIN = np.average(hdu[1].data['BMIN'])/3600  # degrees
# BPA = np.average(hdu[1].data['BPA'])  # Position angle in degrees

# pixel_size = header['CDELT1']
# Dfreq    = header['CDELT3']
# restfreq = header['CRVAL3']

# BeamArea     = np.pi *BMIN*BMAJ/pixel_size**2/(4*np.log(2))   # pixels per beam 
# cube         = cube[0]/BeamArea*1e3                           # mJy/pixel

# mask = ellipsoid_mask(cube[0], 
#                       len(cube[0])//2, 
#                       len(cube[0])//2, 
#                       BMIN/pixel_size, 
#                       BMAJ/pixel_size, 
#                       np.deg2rad(BPA))


# xaxis      = (np.arange(len(cube))*Dfreq + restfreq)/1e9    
# slp        = np.nansum(cube[:, mask], axis = 1)
# normal_std = np.nanstd(cube[:,~mask], axis = 1) * np.sqrt(np.sum(mask))
# mask = ellipsoid_mask(cube[0], 
#                       len(cube[0])//2, 
#                       len(cube[0])//2, 
#                       BMIN/pixel_size, 
#                       BMAJ/pixel_size, 
#                       np.deg2rad(BPA))

# obj                           = SLP(cube, mask, amount = 1000, visualize=False)
# bootstrap_std, bootstrap_mean = obj.execute()

# # Visualize SLP
# # -----------------
# fig, ax = plt.subplots(2,1, sharex=True, figsize=(8,6), gridspec_kw={'height_ratios': [2, 1]})

# ax[0].step(xaxis, slp/bootstrap_std, label = 'Central Beam', c = 'C0')
# ax[0].axhline(0, c='gray', ls='--')
# ax[0].set_xlim(xmin = xaxis[0], xmax = xaxis[-1])
# ax[0].set_ylim(ymin =-2.5, ymax = 3.5)
# ax[0].set_ylabel('SNR')
# ax[0].legend(loc = 1, frameon=False)
# ax[0].axvline(254.35, c = 'k')

# # ax[1].step(xaxis, normal_std, label='conventional image')
# ax[1].step(xaxis, bootstrap_std, label='bootstrap', c = 'C1')
# ax[1].axhline(np.nanstd(slp), label='std spectral', c='k', alpha = 0.6)    

# ax[1].set_xlim(xmin = xaxis[0], xmax = xaxis[-1])
# ax[1].set_ylim(ymin = 0.15, ymax = 0.25)
# ax[1].set_xlabel('Freq [GHz]')
# ax[1].set_ylabel('mJy')
# ax[1].legend(loc = 1, frameon=False)

# plt.tight_layout()
# plt.show()

# Jacked version

In [ ]:
plot_fits_image_with_wcs(fits_file, 4, (3.498985, -30.324767),  (index-5, index+5))  # Replace with your FITS file path, desired box size in arcseconds, and center coordinates (RA, DEC)
direc = './output/C7/model/'
fits_files = os.listdir(direc)

In [ ]:
for f in fits_files:
    if f.endswith('.fits'):
        plot_fits_image_with_wcs(direc+f, 4, (3.498985, -30.324767),  (index-5, index+5))  # Replace with your FITS file path, desired box size in arcseconds, and center coordinates (RA, DEC)

# Line Finder